# BASIN moveout plot notebooks
This notebook makes moveout plots of the stacked cross correlations between broadbands and nodal stations. It is highly inspired by Julian Schmitt's work, modified by Marine Denolle (mdenolle@uw.edu)

In [1]:
# Packages - set GR as plotting backend 
#include("/Users/julianschmitt/Documents/Schoolwork/Seismology/SeisCore.jl/src/SeisCore.jl")
using Pkg, SeisIO, SeisNoise, Plots, Dates, CSV, DataFrames, Distributed, JLD2, Statistics, PyCall, Glob, StructArrays, ColorSchemes, Plots.PlotUtils, HDF5, AbstractFFTs
include("../../src/process.jl")
include("../src/utils.jl")
ENV["GR"] = ""
Pkg.build("GR")
loadcolorscheme(:cm_maxamp,ColorSchemes.gist_heat.colors[end-50:-1:1], "maxamp color", "for waveform plot");

┌ Warning: Error requiring `HDF5` from `Plots`
│   exception = (LoadError("/Users/marinedenolle/.julia/packages/Plots/tXtrW/src/backends/hdf5.jl", 36, UndefVarError(:Group)), Union{Ptr{Nothing}, Base.InterpreterIP}[Ptr{Nothing} @0x000000010e4bfb1f, Ptr{Nothing} @0x000000010e5693cf, Ptr{Nothing} @0x000000010e56a57b, Ptr{Nothing} @0x000000010e568ae6, Ptr{Nothing} @0x000000010e5690dc, Base.InterpreterIP in top-level CodeInfo for Plots._hdf5_implementation at statement 4, Ptr{Nothing} @0x000000010e58415f, Ptr{Nothing} @0x000000010e5842f5, Ptr{Nothing} @0x000000010e583f02, Ptr{Nothing} @0x000000010e583f02, Ptr{Nothing} @0x000000010e5851d5, Ptr{Nothing} @0x0000000118b367da, Ptr{Nothing} @0x000000010e551160, Ptr{Nothing} @0x0000000118b75f45, Ptr{Nothing} @0x0000000118701c85, Ptr{Nothing} @0x000000010e551160, Ptr{Nothing} @0x000000010e55df4a, Ptr{Nothing} @0x000000014dfe4faa, Ptr{Nothing} @0x000000010e551160, Ptr{Nothing} @0x000000010e56a59f, Ptr{Nothing} @0x000000010e568ae6, Ptr{Nothing} @0x0

In [ ]:
# metaparams for plots
n_derivatives = 2
n_deriv_names = ["Raw","PGD","PDV"]
#plot_sorttype = "Latitude" # other options include "Longitude" and (soon) "Distance" (in progress...) 
years = [2017]#, 2018, 2019]
components = ["ZZ"]#["EE","EN","EZ","NE", "NN", "NZ","ZE","ZN", "ZZ"]
source_stations = ["SVD"]#["CHN","CJM","DEV","IPT","LPC","SNO","SVD","TA2"]
rootdir = "/Users/marinedenolle/BASIN" # chose root directory for plots
lw, scale, frequency_plots = 0.5, 3, [[0.1,0.2],[0.15, 0.25],[0.2,0.3],[0.1,0.35]]
locations = DataFrame(CSV.File("/Users/marinedenolle/BASIN/C4-Project.jl/docs/modified_nodal.csv"));